In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
from torchvision import transforms
import time
from tqdm.autonotebook import tqdm
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from inspect import getfullargspec,signature

import matplotlib.pyplot as plt
import numpy as np

from torch.utils.data import Dataset
from torchvision import datasets

import torch as ch
from robustness.datasets import CINIC
ds = CINIC('/tmp/cinic')

from sklearn import metrics

%matplotlib inline
%config InlineBackend.figure_format = 'svg'


In [2]:
############################
# Use pytorch pretrained Resnet18
#######################

In [3]:
        
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")
    
# load STD model 
############################
class CinicResNet(nn.Module):
  def __init__(self, in_channels=3):
    super(CinicResNet, self).__init__()

    # Load a pretrained resnet model from torchvision.models in Pytorch
    self.model = models.resnet18(pretrained=False)

    # Change the input layer to take Grayscale image, instead of RGB images.  **************
    # Hence in_channels is set as 1 or 3 respectively
    # original definition of the first layer on the ResNet class
    # self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
    self.model.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
    
    # Change the output layer to output 10 classes instead of 1000 classes
    num_ftrs = self.model.fc.in_features
    self.model.fc = nn.Linear(num_ftrs, 10)

  def forward(self, x):
    return self.model(x)

# load STD model 
model_std = CinicResNet().to(device)

# params you need to specify:
epochs = 120
batch_size = 128
print(model_std)
model_state_dict = ch.load("./CinicSTDNet")
model_std.load_state_dict(model_state_dict)
model_std.to(device)

CinicResNet(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_r

CinicResNet(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_r

In [4]:
### Initiate ADV model 
from robustness.model_utils import make_and_restore_model
model_adv, _ = make_and_restore_model(arch='resnet18', dataset=ds,
              resume_path= '/home/ashraf.haddad/mlpr/CINIC-Store-ADV/checkpoint.pt.best') #'/tmp/35daedae-1b39-4941-ad08-8bd6459c1bd8/checkpoint.pt.best')
            
model_adv.eval()
pass

=> loading checkpoint '/home/ashraf.haddad/mlpr/CINIC-Store-ADV/checkpoint.pt.best'
=> loaded checkpoint '/home/ashraf.haddad/mlpr/CINIC-Store-ADV/checkpoint.pt.best' (epoch 56)


In [10]:
#only load the validation set
ATTACK_EPS = 0.5
ATTACK_STEPSIZE = 0.1
ATTACK_STEPS = 10
NUM_WORKERS = 8
BIG_BATCH_SIZE = 600 #128

_, test_loader = ds.make_loaders(workers=NUM_WORKERS, batch_size=BIG_BATCH_SIZE)
_, (im, label) = next(enumerate(test_loader)) 
im,label = im.to(device),label.to(device)

==> Preparing dataset cinic..


In [11]:
kwargs = {
    'constraint':'2', # use L2-PGD
    'eps': ATTACK_EPS, # L2 radius around original image
    'step_size': ATTACK_STEPSIZE,
    'iterations': ATTACK_STEPS,
    'do_tqdm': True,
}
label.size()

torch.Size([600])

In [12]:
print(torch.cuda.is_available())
torch.cuda.empty_cache() 

True


In [13]:
## Generate adv examples 

#the model can generate adv by applying max l-infinity loss
_, im_adv = model_adv(im, label, make_adv=True, **kwargs)

im_adv_transformed = transforms.Resize((224, 224)).forward(im_adv)
print(im_adv[0].size())
print(im_adv_transformed[0].size())




Current loss: 1.2181929349899292: 100%|█████████| 10/10 [00:04<00:00,  2.02it/s]


torch.Size([3, 32, 32])
torch.Size([3, 224, 224])


In [14]:

# predict and measure the accuracy 

# Get predicted labels for adversarial examples using 
#im_split = ch.split(im_adv_transformed, batch_size)
#im_adv_transformed.size()
#len
pred = model_std(im_adv_transformed)
#pred = [model_std(sp) for sp in im_split ]
label_pred = ch.argmax(pred, dim=1)
from sklearn import metrics

print("Accuracy:",metrics.accuracy_score(label.cpu() ,label_pred.cpu()))


Accuracy: 0.14
